In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Move folder from drive and install requirements


In [ ]:
!unzip /content/drive/MyDrive/HIISS/Causal-Unsupervised-Segmentation.zip &> /dev/null
!mv /content/content/Causal-Unsupervised-Segmentation /content
!rm -r /content/content

In [ ]:
!pip install -r /content/Causal-Unsupervised-Segmentation/requirements.txt &> /dev/null
!pip install timm==0.9.5 &> /dev/null
!pip install git+https://github.com/lucasb-eyer/pydensecrf.git &> /dev/null

# Load pirelli datasets

In [ ]:
!unzip /content/drive/MyDrive/HIISS/datasets/Pirelli_3cat_strided.zip -d /content &> /dev/null

In [ ]:
!unzip /content/drive/MyDrive/HIISS/datasets/Pirelli_4cat_strided.zip -d /content &> /dev/null

In [ ]:
!unzip /content/drive/MyDrive/HIISS/datasets/Pirelli_3cat_no_stride.zip -d /content &> /dev/null

In [ ]:
!unzip /content/drive/MyDrive/HIISS/datasets/Pirelli_4cat_no_stride.zip -d /content &> /dev/null

In [ ]:
!unzip /content/drive/MyDrive/HIISS/datasets/Pirelli_unsupervised_no_stride.zip -d /content &> /dev/null

# Load Dino checkpoint


In [ ]:
!mkdir checkpoint
!wget https://dl.fbaipublicfiles.com/dino/dino_deitsmall8_pretrain/dino_deitsmall8_pretrain.pth &> /dev/null
!mv /content/dino_deitsmall8_pretrain.pth /content/checkpoint/dino_vit_small_8.pth

# Load CAUSE checkpoint

In [ ]:
!rm -r /content/CAUSE

In [ ]:
!cp /content/drive/MyDrive/HIISS/checkpoints/mediator_unsupervised_no_stride.zip .
!unzip mediator_unsupervised_no_stride.zip -d ./CAUSE

Archive:  mediator_unsupervised_no_stride.zip
   creating: ./CAUSE/content/CAUSE/
   creating: ./CAUSE/content/CAUSE/pirelli/
   creating: ./CAUSE/content/CAUSE/pirelli/modularity/
   creating: ./CAUSE/content/CAUSE/pirelli/modularity/dino_vit_small_8/
   creating: ./CAUSE/content/CAUSE/pirelli/modularity/dino_vit_small_8/2048/
  inflating: ./CAUSE/content/CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy  


In [ ]:
!cp -r /content/CAUSE/content/CAUSE .
!rm -r /content/CAUSE/content
!rm -r /content/mediator_unsupervised_no_stride.zip

# Crop the datasets


In [ ]:
!python ./Causal-Unsupervised-Segmentation/crop_dataset.py  --data_dir "Pirelli_4cat" --dataset "pirelli" --crop_type "super"

Device = cuda:0
100% 130/130 [00:25<00:00,  5.14it/s]


In [ ]:
!python ./Causal-Unsupervised-Segmentation/crop_dataset.py  --data_dir "Pirelli_3cat" --dataset "pirelli" --crop_type "super"

Device = cuda:0
100% 130/130 [00:25<00:00,  5.11it/s]


In [ ]:
!python ./Causal-Unsupervised-Segmentation/crop_dataset.py --data_dir "Pirelli_unsupervised" --dataset "pirelli" --crop_type "super"

Device = cuda:0
100% 1335/1335 [10:00<00:00,  2.22it/s]


# Train the mediator

In [ ]:
!rm -r CAUSE # Remove to retrain again

rm: cannot remove 'CAUSE': No such file or directory


In [ ]:
!python ./Causal-Unsupervised-Segmentation/train_mediator.py \
        --data_dir "Pirelli_unsupervised" --dataset "pirelli" \
        --ckpt "/content/checkpoint/dino_vit_small_8.pth"\
        --gpu 0 --epoch 3

------------------Configurations------------------
epoch: 3
distributed: True
load_segment: False
load_cluster: False
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
data_dir: Pirelli_3cat
dataset: pirelli
ckpt: /content/checkpoint/dino_vit_small_8.pth
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 384
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
No File Exists!!
-------------TRAIN EPOCH: 1-------------
[Train]: 100% 407/407 [01:52<00:00,  3.61it/s]
-------------TRAIN EPOCH: 2-------------
[Train]: 100% 407/407 [01:51<00:00,  3.66it/s]
-------------TRAIN EPOCH: 3-------------
[Train]: 100% 407/407 [01:51<00:00,  3.66it/s]


In [ ]:
!zip -r mediator_unsupervised_no_stride.zip /content/CAUSE

  adding: content/CAUSE/ (stored 0%)
  adding: content/CAUSE/pirelli/ (stored 0%)
  adding: content/CAUSE/pirelli/modularity/ (stored 0%)
  adding: content/CAUSE/pirelli/modularity/dino_vit_small_8/ (stored 0%)
  adding: content/CAUSE/pirelli/modularity/dino_vit_small_8/2048/ (stored 0%)
  adding: content/CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy (deflated 10%)


In [ ]:
!mv /content/mediator_unsupervised_no_stride.zip /content/drive/MyDrive/HIISS/checkpoints

# Train the transformer

## 4 class

In [ ]:
!python ./Causal-Unsupervised-Segmentation/train_front_door_tr.py \
          --data_dir "Pirelli_4cat" --dataset "pirelli" \
          --ckpt "/content/checkpoint/dino_vit_small_8.pth" \
          --gpu 0 --epoch 5 --batch_size 16

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_4cat
dataset: pirelli
ckpt: /content/checkpoint/dino_vit_small_8.pth
epoch: 5
distributed: False
load_segment: False
load_cluster: False
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
4
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
-------------TRAIN EPOCH: 1-------------
[Train] Loss: 7.26=6.82+0.45 ACC: 85.6%: 100% 407/407 [03:49<00:00,  1.77it/s]
-------------TEST EPOCH: 1-------------
[TEST] Acc (Linear): 92.0%: 100% 3/3 [00:01<00:00,  2.43it/s]
-----------------TEST Epoch 0: SAVING CHECKPOINT IN CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth-----------------
-------------TRAIN EPOCH: 2----------

## 3 class

In [ ]:
!python ./Causal-Unsupervised-Segmentation/train_front_door_tr.py \
          --data_dir "Pirelli_3cat" --dataset "pirelli" \
          --ckpt "/content/checkpoint/dino_vit_small_8.pth" \
          --gpu 0 --epoch 4 --batch_size 16

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_3cat
dataset: pirelli
ckpt: /content/checkpoint/dino_vit_small_8.pth
epoch: 5
distributed: False
load_segment: False
load_cluster: False
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
3
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
-------------TRAIN EPOCH: 1-------------
[Train] Loss: 7.17=6.82+0.35 ACC: 87.1%: 100% 407/407 [03:48<00:00,  1.78it/s]
-------------TEST EPOCH: 1-------------
[TEST] Acc (Linear): 92.8%: 100% 3/3 [00:01<00:00,  2.45it/s]
-----------------TEST Epoch 0: SAVING CHECKPOINT IN CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth-----------------
-------------TRAIN EPOCH: 2----------

#Fine tuning

## 4 class

In [ ]:
!python /content/Causal-Unsupervised-Segmentation/fine_tuning_tr.py \
          --data_dir "Pirelli_4cat" --dataset "pirelli" \
          --ckpt "/content/checkpoint/dino_vit_small_8.pth"  \
          --gpu 0 --epoch 3 --batch_size 16 --load_segment True \
          --load_cluster False

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_4cat
dataset: pirelli
ckpt: /content/checkpoint/dino_vit_small_8.pth
epoch: 3
distributed: True
load_segment: True
load_cluster: False
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
4
[Segment] CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth loaded
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
-------------TRAIN EPOCH: 1-------------
[Train] Loss: 0.00=0.10-0.10 ACC: 96.3%: 100% 407/407 [02:46<00:00,  2.45it/s]
-------------TEST EPOCH: 1-------------
[TEST] Acc (Linear): 97.6% | [mIoU]: 83.6, [mAP]: 87.2, [Acc]: 96.2, : 100% 3/3 [00:01<00:00,  2.29it/s]
-----------------TEST Epoch 0: SAVING CHECKPOINT

In [ ]:
!python /content/Causal-Unsupervised-Segmentation/fine_tuning_tr.py \
          --data_dir "Pirelli_4cat" --dataset "pirelli" \
          --ckpt "/content/checkpoint/dino_vit_small_8.pth"  \
          --gpu 0 --epoch 1 --batch_size 16 --load_segment True \
          --load_cluster False

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_4cat
dataset: pirelli
ckpt: /content/checkpoint/dino_vit_small_8.pth
epoch: 1
distributed: True
load_segment: True
load_cluster: False
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
4
[Segment] CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth loaded
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
-------------TRAIN EPOCH: 1-------------
[Train] Loss: -0.02=0.05-0.07 ACC: 97.9%: 100% 407/407 [02:45<00:00,  2.46it/s]
-------------TEST EPOCH: 1-------------
[TEST] Acc (Linear): 97.9% | [mIoU]: 39.3, [mAP]: 47.8, [Acc]: 69.5, : 100% 3/3 [00:01<00:00,  2.19it/s]
-----------------TEST Epoch 0: SAVING CHECKPOIN

## 3 class

In [ ]:
!python /content/Causal-Unsupervised-Segmentation/fine_tuning_tr.py \
          --data_dir "Pirelli_3cat" --dataset "pirelli" \
          --ckpt "/content/checkpoint/dino_vit_small_8.pth"  \
          --gpu 0 --epoch 3 --batch_size 16 --load_segment True \
          --load_cluster False

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_3cat
dataset: pirelli
ckpt: /content/checkpoint/dino_vit_small_8.pth
epoch: 3
distributed: True
load_segment: True
load_cluster: False
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
3
[Segment] CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth loaded
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
-------------TRAIN EPOCH: 1-------------
[Train] Loss: 0.05=0.11-0.06 ACC: 96.3%: 100% 407/407 [02:45<00:00,  2.46it/s]
-------------TEST EPOCH: 1-------------
[TEST] Acc (Linear): 97.8% | [mIoU]: 58.3, [mAP]: 60.0, [Acc]: 92.1, : 100% 3/3 [00:01<00:00,  1.99it/s]
-----------------TEST Epoch 0: SAVING CHECKPOINT

In [ ]:
!python /content/Causal-Unsupervised-Segmentation/fine_tuning_tr.py \
          --data_dir "Pirelli_3cat" --dataset "pirelli" \
          --ckpt "/content/checkpoint/dino_vit_small_8.pth"  \
          --gpu 0 --epoch 1 --batch_size 16 --load_segment True \
          --load_cluster False

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_3cat
dataset: pirelli
ckpt: /content/checkpoint/dino_vit_small_8.pth
epoch: 1
distributed: True
load_segment: True
load_cluster: False
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
port: 12355
grid: True
num_codebook: 2048
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
3
[Segment] CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth loaded
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
-------------TRAIN EPOCH: 1-------------
[Train] Loss: -0.08=0.05-0.14 ACC: 97.9%: 100% 407/407 [02:45<00:00,  2.46it/s]
-------------TEST EPOCH: 1-------------
[TEST] Acc (Linear): 97.6% | [mIoU]: 89.0, [mAP]: 92.9, [Acc]: 97.8, : 100% 3/3 [00:01<00:00,  2.29it/s]
-----------------TEST Epoch 0: SAVING CHECKPOIN

# Test the model

## 4 class

In [ ]:
!python /content/Causal-Unsupervised-Segmentation/test_tr.py \
        --data_dir "Pirelli_4cat" --dataset "pirelli" \
        --ckpt "/content/checkpoint/dino_vit_small_8.pth" \
        --load_segment True --load_cluster True

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_4cat
dataset: pirelli
port: 12355
ckpt: /content/checkpoint/dino_vit_small_8.pth
distributed: False
load_segment: True
load_cluster: True
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
num_codebook: 2048
just_imgs: False
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
4
[Segment] CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth loaded
[Cluster] CAUSE/pirelli/dino_vit_small_8/2048/cluster_tr.pth loaded
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
# of Parameters: 3.59(M)
Test without CRF
[TEST] Acc (Linear): 97.9% | [mIoU]: 39.3, [mAP]: 47.8, [Acc]: 69.5, : 100% 3/3 [00:01<00:00,  1.65it/s]
OrderedDict([('mIoU', tensor([59.9372, 93.7674,  3.4182,  0.0000], device='cuda:0')), ('mAP', tensor(

## 3 class

In [ ]:
!python /content/Causal-Unsupervised-Segmentation/test_tr.py \
        --data_dir "Pirelli_3cat" --dataset "pirelli" \
        --ckpt "/content/checkpoint/dino_vit_small_8.pth" \
        --load_segment True --load_cluster True

------------------Configurations------------------
NAME_TAG: CAUSE-TR
data_dir: Pirelli_3cat
dataset: pirelli
port: 12355
ckpt: /content/checkpoint/dino_vit_small_8.pth
distributed: False
load_segment: True
load_cluster: True
train_resolution: 320
test_resolution: 320
batch_size: 16
num_workers: 0
gpu: 0
num_codebook: 2048
just_imgs: False
reduced_dim: 90
projection_dim: 2048
dim: 384
num_queries: 1600
-------------------------------------------------
_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])
3
[Segment] CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth loaded
[Cluster] CAUSE/pirelli/dino_vit_small_8/2048/cluster_tr.pth loaded
Modularity CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy loaded
# of Parameters: 3.59(M)
Test without CRF
[TEST] Acc (Linear): 97.6% | [mIoU]: 89.0, [mAP]: 92.9, [Acc]: 97.8, : 100% 3/3 [00:01<00:00,  1.62it/s]
OrderedDict([('mIoU', tensor([97.4479, 96.3618, 73.2250], device='cuda:0')), ('mAP', tensor([98.8375,

# Save Results and checkpoints

## 4 class

In [ ]:
!zip -r results_med-unsup_train-sup_4classes.zip ./results &> /dev/null

In [ ]:
!mv /content/results_med-unsup_train-sup_4classes.zip /content/drive/MyDrive/HIISS/results

In [ ]:
!zip -r CAUSE_checkpoint_med-unsup_train-sup_4classes.zip /content/CAUSE

  adding: content/CAUSE/ (stored 0%)
  adding: content/CAUSE/pirelli/ (stored 0%)
  adding: content/CAUSE/pirelli/dino_vit_small_8/ (stored 0%)
  adding: content/CAUSE/pirelli/dino_vit_small_8/2048/ (stored 0%)
  adding: content/CAUSE/pirelli/dino_vit_small_8/2048/cluster_tr.pth (deflated 9%)
  adding: content/CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth (deflated 7%)
  adding: content/CAUSE/pirelli/modularity/ (stored 0%)
  adding: content/CAUSE/pirelli/modularity/dino_vit_small_8/ (stored 0%)
  adding: content/CAUSE/pirelli/modularity/dino_vit_small_8/2048/ (stored 0%)
  adding: content/CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy (deflated 9%)


In [ ]:
!mv /content/CAUSE_checkpoint_med-unsup_train-sup_4classes.zip /content/drive/MyDrive/HIISS/checkpoints

## 3 class

In [ ]:
!zip -r results_med-unsup_train-sup_3classes.zip ./results &> /dev/null

In [ ]:
!mv /content/results_med-unsup_train-sup_3classes.zip /content/drive/MyDrive/HIISS/results

In [ ]:
!zip -r CAUSE_checkpoint_med-unsup_train-sup_3classes.zip /content/CAUSE

  adding: content/CAUSE/ (stored 0%)
  adding: content/CAUSE/pirelli/ (stored 0%)
  adding: content/CAUSE/pirelli/modularity/ (stored 0%)
  adding: content/CAUSE/pirelli/modularity/dino_vit_small_8/ (stored 0%)
  adding: content/CAUSE/pirelli/modularity/dino_vit_small_8/2048/ (stored 0%)
  adding: content/CAUSE/pirelli/modularity/dino_vit_small_8/2048/modular.npy (deflated 10%)
  adding: content/CAUSE/pirelli/dino_vit_small_8/ (stored 0%)
  adding: content/CAUSE/pirelli/dino_vit_small_8/2048/ (stored 0%)
  adding: content/CAUSE/pirelli/dino_vit_small_8/2048/segment_tr.pth (deflated 7%)
  adding: content/CAUSE/pirelli/dino_vit_small_8/2048/cluster_tr.pth (deflated 10%)


In [ ]:
!mv /content/CAUSE_checkpoint_med-unsup_train-sup_3classes.zip /content/drive/MyDrive/HIISS/checkpoints